In this model i have used The Complete Works of William Shakespeare by William Shakespeare plain text UTF-8

to download this text data go to https://www.gutenberg.org/ebooks/100 and find plain text UTF-8 and save it as .txt and upload it into google colab environment

as there was time constrain i used smaller set from the dataset and the accuracy was low but given enough time i can fine tune the model and ruduce the training time and increace the model accuracy  

In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.2 MB/s eta 0:00:00


In [9]:
# Import statements and parameters
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import re




In [12]:
FILE_PATH = 'shakespeare.txt'
MAX_TEXT_LENGTH = 50000

try:
    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        text_data = f.read(MAX_TEXT_LENGTH) # Load only a part of the text
except FileNotFoundError:
    print(f"Error: The file '{FILE_PATH}' was not found. Please upload it to your Colab session.")
    text_data = "This is a dummy text if the file is not found. Please upload your dataset."

# Inspect the data (first 1000 characters) ---
if text_data:
    print("--- First 1000 characters of the dataset ---")
    print(text_data[:1000])
    print(f"\n--- Total characters in dataset: {len(text_data)} ---")

--- First 1000 characters of the dataset ---
﻿The Project Gutenberg eBook of The Complete Works of William Shakespeare
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Release date: January 1, 1994 [eBook #100]
                Most recently updated: October 29, 2024

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***
The Complete Works of William Shakespeare

by William Shakespeare




                    Contents

    THE SONNETS
    ALL’S WELL THAT ENDS WELL

In [13]:
# Preprocess the text
def preprocess_text(text):
    processed_text = text.lower()
    processed_text = re.sub(r'^\s*\*\*\* START OF THIS PROJECT GUTENBERG EBOOK.*? \*\*\*\s*', '', processed_text, flags=re.IGNORECASE | re.DOTALL)
    processed_text = re.sub(r'\s*\*\*\* END OF THIS PROJECT GUTENBERG EBOOK.*? \*\*\*.*$', '', processed_text, flags=re.IGNORECASE | re.DOTALL)
    processed_text = re.sub(r'produced by .*?\n', '', processed_text, flags=re.IGNORECASE)
    processed_text = re.sub(r'\[illustration.*?\]', '', processed_text, flags=re.IGNORECASE)
    allowed_punctuation = '.\n'
    unwanted_chars_pattern = r"[^a-z0-9\s" + re.escape(allowed_punctuation) + r"]"
    processed_text = re.sub(unwanted_chars_pattern, ' ', processed_text)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    return processed_text

if 'text_data' in locals() and text_data:
    cleaned_text = preprocess_text(text_data)
    print("--- First 500 characters of cleaned text ---")
    print(cleaned_text[:500])
    print(f"\n--- Total characters in cleaned dataset: {len(cleaned_text)} ---")
else:
    print("Text data not loaded. Please ensure the file is uploaded and Cell 2 ran successfully.")
    cleaned_text = ""

--- First 500 characters of cleaned text ---
the project gutenberg ebook of the complete works of william shakespeare this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever. you may copy it give it away or re use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org. if you are not located in the united states you will have to check the laws of the country where you are located before usin

--- Total characters in cleaned dataset: 46629 ---


In [14]:
#Tokenization and sequence generation
if 'cleaned_text' in locals() and cleaned_text:
    chars = sorted(list(set(cleaned_text)))
    char_to_int = {char: i for i, char in enumerate(chars)}
    int_to_char = {i: char for i, char in enumerate(chars)}

    n_chars = len(cleaned_text)
    n_vocab = len(chars)

    print(f"--- Total unique characters (vocab size): {n_vocab} ---")
    if n_vocab > 0:
        print(f"--- Character mapping (first 10): {list(char_to_int.items())[:min(10, n_vocab)]} ---")
    else:
        print("--- No characters found in cleaned_text. Vocabulary is empty. ---")

    seq_length = 50  # Shorter sequence length for faster training
    step = 1

    input_sequences = []
    output_chars = []

    if n_vocab > 0 and n_chars > seq_length:
        for i in range(0, n_chars - seq_length, step):
            current_seq_text = cleaned_text[i : i + seq_length]
            next_char_text = cleaned_text[i + seq_length]
            input_sequences.append([char_to_int[char] for char in current_seq_text])
            output_chars.append(char_to_int[next_char_text])

        n_patterns = len(input_sequences)
        print(f"--- Total input-output patterns: {n_patterns} ---")

        if n_patterns > 0:
            print("--- Example Input Sequence (first actual sequence): ---")
            print("Text:", cleaned_text[0 : seq_length])
            print("Ints:", input_sequences[0])
            print("--- Example Output Character (for first actual sequence): ---")
            print("Text:", cleaned_text[seq_length])
            print("Int:", output_chars[0])
        else:
            print("No patterns generated. Check seq_length or cleaned_text content.")
    elif n_vocab == 0:
        print("Vocabulary size is 0. Cannot create sequences.")
        n_patterns = 0
    else:
        print(f"Cleaned text length ({n_chars}) is not greater than sequence length ({seq_length}). Cannot create sequences.")
        n_patterns = 0
else:
    print("Cleaned text is empty or not defined. Please check previous cells.")
    n_patterns = 0
    input_sequences = []
    output_chars = []
    n_vocab = 0
    seq_length = 100
    char_to_int = {}
    int_to_char = {}

--- Total unique characters (vocab size): 38 ---
--- Character mapping (first 10): [(' ', 0), ('.', 1), ('0', 2), ('1', 3), ('2', 4), ('3', 5), ('4', 6), ('5', 7), ('6', 8), ('7', 9)] ---
--- Total input-output patterns: 46579 ---
--- Example Input Sequence (first actual sequence): ---
Text: the project gutenberg ebook of the complete works 
Ints: [31, 19, 16, 0, 27, 29, 26, 21, 16, 14, 31, 0, 18, 32, 31, 16, 25, 13, 16, 29, 18, 0, 16, 13, 26, 26, 22, 0, 26, 17, 0, 31, 19, 16, 0, 14, 26, 24, 27, 23, 16, 31, 16, 0, 34, 26, 29, 22, 30, 0]
--- Example Output Character (for first actual sequence): ---
Text: o
Int: 26


In [15]:
# Prepare data for the Keras embedding model
if ('n_patterns' in locals() and n_patterns > 0 and
    'input_sequences' in locals() and 'output_chars' in locals() and
    'n_vocab' in locals() and n_vocab > 0 and 'seq_length' in locals()):

    X_train = np.array(input_sequences)
    y_train = to_categorical(output_chars, num_classes=n_vocab)

    print("--- Data Shapes for Model Training ---")
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)

else:
    print("Skipping model data preparation: Not enough patterns or necessary variables are missing.")
    print("Ensure Cell 4 (Tokenization and Sequence Generation) ran successfully.")
    X_train = None
    y_train = None

--- Data Shapes for Model Training ---
X_train shape: (46579, 50)
y_train shape: (46579, 38)


In [16]:
#Model design and compilation
if 'n_vocab' in locals() and 'seq_length' in locals():
    embedding_dim = 50 # Reduced embedding dimension
    lstm_units = 128 # Reduced LSTM units

    model = Sequential([
        Embedding(input_dim=n_vocab, output_dim=embedding_dim, input_shape=(seq_length,)),
        LSTM(lstm_units),
        Dropout(0.2),
        Dense(n_vocab, activation='softmax')
    ])

    model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    print("\n--- Model Summary ---")
    model.summary()

else:
    print("Skipping model design and compilation: 'n_vocab' or 'seq_length' not defined.")
    model = None


--- Model Summary ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 50, 50)         │         1,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,450 (384.57 KB)

 Trainable params: 98,450 (384.57 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
#Model training
if 'model' in locals() and model is not None and \
    'X_train' in locals() and X_train is not None and \
    'y_train' in locals() and y_train is not None:
    print("--- Starting Model Training ---")
    epochs = 8 # Reduced number of epochs for faster training
    batch_size = 128 # Adjust based on memory and dataset size

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size) # Removed validation_split for faster training
    print("--- Model Training Complete ---")

    # You can then save the model, plot history, and generate text
    model.save('shakespeare_char_lstm_model.keras')
else:
    print("Model, X_train, or y_train not available. Cannot start training.")
    print("Please ensure Cells 1-6 ran successfully.")

--- Starting Model Training ---
Epoch 1/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 156ms/step - accuracy: 0.4545 - loss: 1.8324
Epoch 2/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 155ms/step - accuracy: 0.4575 - loss: 1.7971
Epoch 3/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 155ms/step - accuracy: 0.4731 - loss: 1.7530
Epoch 4/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 156ms/step - accuracy: 0.4818 - loss: 1.7183
Epoch 5/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 155ms/step - accuracy: 0.4834 - loss: 1.6904
Epoch 6/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 155ms/step - accuracy: 0.4931 - loss: 1.6639
Epoch 7/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 57s 156ms/step - accuracy: 0.4985 - loss: 1.6456
Epoch 8/8
364/364 ━━━━━━━━━━━━━━━━━━━━ 56s 155ms/step - accuracy: 0.5056 - loss: 1.6143
--- Model Training Complete ---


In [20]:
#Text Generation

if 'model' in locals() and model is not None and \
   'int_to_char' in locals() and 'char_to_int' in locals() and \
   'input_sequences' in locals() and len(input_sequences) > 0 and \
   'seq_length' in locals() and 'n_vocab' in locals():

    print("--- Text Generation ---")

    # Select a random seed sequence
    start_index = np.random.randint(0, len(input_sequences) - 1)
    seed_sequence = input_sequences[start_index]
    generated_text = [int_to_char[index] for index in seed_sequence]
    print("--- Generating text with seed: ---\n", "".join(generated_text))

    # Parameters for text generation
    num_chars_to_generate = 100
    print("\n--- Generating {} characters... ---".format(num_chars_to_generate))

    # Generate text
    for _ in range(num_chars_to_generate):
       n
        input_array = np.reshape(seed_sequence, (1, seq_length))

        # Make a prediction
        prediction = model.predict(input_array, verbose=0)
        predicted_index = np.argmax(prediction)
        predicted_char = int_to_char[predicted_index]

        # Append the predicted character to the generated text and update the seed
        generated_text.append(predicted_char)
        seed_sequence.append(predicted_index)
        seed_sequence = seed_sequence[1:] # Remove the first character to maintain sequence length

    print("\n--- Generated Text: ---")
    print("".join(generated_text))

else:
    print("Model or necessary variables not found. Please ensure all previous cells have been run successfully.")

--- Text Generation ---
--- Generating text with seed: ---
 ost common grow. 70 that thou art blamed shall not

--- Generating 100 characters... ---

--- Generated Text: ---
ost common grow. 70 that thou art blamed shall not the self the self the self the self the self the self the self the self the self the self the self 
